In [1]:
from aocd import get_data

# Day 10: Hoof It

## Part One

A map shows the height at each position, with values ranging from 0 (lowest) to 9 (highest). 
Fill in hiking trails that start at height 0 and end at height 9, always increasing by exactly 1 at each step (no diagonal movement allowed).

**Key Elements:**
- **Trailhead:** Any position with height 0 that can start a hiking trail
- **Validity:** Starts at height 0, ends at height 9, and increases by exactly 1 at each step (up, down, left, or right)
- **Trailhead Score:** The number of positions with height 9 that are reachable via a hiking trail

**Steps:**
1. **Identify Trailheads:** Find all positions with height 0 (trailheads)
2. **Trace Hiking Trails:** From each trailhead, trace valid hiking trails that only increase by 1 and do not allow diagonal steps
3. **Calculate Scores:** For each trailhead, calculate how many 9-height positions are reachable
4. **Sum of Scores:** Sum the scores of all trailheads to find the final answer


**Challenge:**
Implement an algorithm to find and trace these hiking trails from each trailhead and compute the sum of the trailhead scores.







This trailhead has a score of 2:
```
...0...
...1...
...2...
6543456
7.....7
8.....8
9.....9
```
The positions marked `.` are impassable tiles to simplify this example; they do not appear on your actual topographic map.

Here's a larger example:

In [2]:
example_input = """
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
"""

This larger example has **9** trailheads.<br/>
Considering the trailheads in reading order, they have scores of 5, 6, 5, 3, 1, 3, 5, 3, and 5. <br/>
Adding these scores together, the sum of the scores of all trailheads is **36**.

In [3]:
import numpy as np

example_map = np.array([list(line) for line in example_input.strip().split("\n")], dtype=int); example_map

array([[8, 9, 0, 1, 0, 1, 2, 3],
       [7, 8, 1, 2, 1, 8, 7, 4],
       [8, 7, 4, 3, 0, 9, 6, 5],
       [9, 6, 5, 4, 9, 8, 7, 4],
       [4, 5, 6, 7, 8, 9, 0, 3],
       [3, 2, 0, 1, 9, 0, 1, 2],
       [0, 1, 3, 2, 9, 8, 0, 1],
       [1, 0, 4, 5, 6, 7, 3, 2]])

In [4]:
start_positions = np.argwhere(example_map==0); start_positions

array([[0, 2],
       [0, 4],
       [2, 4],
       [4, 6],
       [5, 2],
       [5, 5],
       [6, 0],
       [6, 6],
       [7, 1]])

In [5]:
valid_moves = [(-1, 0),(0, 1),(1, 0),(0, -1)]

def is_within_bounds(pos, grid):
    num_rows, num_cols = grid.shape
    return 0 <= pos[0] < num_rows and 0 <= pos[1] < num_cols
    

In [6]:
from collections import deque

def breadth_first_search(map_grid, start_pos):

    start_row, start_col = start_pos
    # double-ended queue allows us to pop from beginning or end of queue
    queue = deque([start_pos]) 
    visited = set([(start_row, start_col)])
    reachable_9s = 0

    while queue:
        r, c = queue.popleft()

        if map_grid[r][c] == 9:
            reachable_9s += 1

        # explore
        for dr, dc in valid_moves:
            nr, nc = r + dr, c + dc
            if is_within_bounds((nr, nc), map_grid) and (nr, nc) not in visited:
                
                if map_grid[nr][nc] == map_grid[r][c] + 1:  # is increment by 1 (what we're looking for)
                    visited.add((nr, nc))
                    queue.append((nr, nc))

    return reachable_9s

def solve_topographic_map(map_grid):
    total_score = 0

    start_positions = np.argwhere(map_grid==0)
    for start_pos in start_positions:
        score = breadth_first_search(map_grid, start_pos)
        total_score += score

    return total_score

In [7]:
result = solve_topographic_map(example_map)
print(f"Sum of all trailhead scores: {result}")

Sum of all trailhead scores: 36


In [8]:
data = get_data(day=10, year=2024)

In [9]:
t_map = np.array([list(map(int, row)) for row in data.splitlines()], dtype=int)

In [10]:
t_map.shape

(57, 57)

In [11]:
print(f"Sum of all trailhead scores: {solve_topographic_map(t_map)}")

Sum of all trailhead scores: 782


## Part Two

There is a second way to measure a trailhead called its rating. A trailhead's rating is the number of distinct hiking trails which begin at that trailhead. Return the sum of all trailhead ratings.

Instead of just counting the number of reachable height-9 positions, you're now counting all possible paths that satisfy the conditions:

- Starts at 0.
- Ends at 9.
- Follows a strict height increase of 1 per step.

This means you're considering all the different ways you can get to a height-9 position, not just the fact that it's reachable.

In [30]:
def get_trailhead_ratings(map_grid, start_pos):

    start_row, start_col = start_pos
    # double-ended queue allows us to pop from beginning or end of queue
    queue = deque([(start_pos, [(start_row, start_col)])])
    visited = set([(start_row, start_col)])
    paths = []  # list to store all valid paths

    while queue:
        (r, c), path = queue.popleft()

        if map_grid[r][c] == 9:
            paths.append(path)

        for dr, dc in valid_moves:
            nr, nc = r + dr, c + dc
            
            if is_within_bounds((nr, nc), map_grid) and (nr, nc) not in path:
                if map_grid[nr][nc] == map_grid[r][c] + 1:  
                    queue.append(((nr, nc), path + [(nr, nc)]))

    return paths

In [31]:
sum_ratings = 0

for start_pos in start_positions:
    all_paths = get_trailhead_ratings(example_map, start_pos)
    print(f"Total Paths: {len(all_paths)}")
    sum_ratings += len(all_paths)



Total Paths: 20
Total Paths: 24
Total Paths: 10
Total Paths: 4
Total Paths: 1
Total Paths: 4
Total Paths: 5
Total Paths: 8
Total Paths: 5


In [32]:
print(sum_ratings)

81


In [34]:
sum_ratings = 0

start_positions = np.argwhere(t_map==0)
for start_pos in start_positions:
    all_paths = get_trailhead_ratings(t_map, start_pos)
    sum_ratings += len(all_paths)

print(sum_ratings)

1694
